# Accessibilité relative des résidus et évolution protéique

Il y a plusieurs étapes dans le traitement des données :

 * obtenir les données pdb par **PISCES**
 * faire tourner **DSSP** pour obtenir l'accessibilité du solvant
 * récupération des séquences homologues par **PSI-BLAST** (pas moins de 70% d'identité)
 * réalignement avec la séquence parente
 * repositionnement des chaines latérales avec **SCWRL4** -> accessibilité du solvant ? 

## PISCES

Sélection de la base de données la plus petite base de [données](http://dunbrack.fccc.edu/Guoli/culledpdb_hh/cullpdb_pc20_res1.6_R0.25_d201103_chains3760.gz)

*Afin de simplifier le traitement on travaille tout d'abord avec un fichier PDB, on résoud les problèmes, une fois que les exceptions sont réglées, on automatise.*

Pour récupérer les fichiers PDB par ligne de commande simplement faire :  
`wget https://files.rscb.org/download/XXXX.pdb`  
Le fichier pdb se créera dans le dossier sous le nom XXXX.pdb avec XXXX l'ID de la protéine.

## DSSP

In [19]:
#imports
from Bio.PDB import PDBParser
from Bio.PDB.DSSP import DSSP

In [20]:
p = PDBParser()
structure = p.get_structure("1A62","../data/1A62.pdb")
model = structure[0]
DSSP = DSSP(model, "../data/1A62.pdb")

In [22]:
for k in DSSP.keys():
    print(DSSP[k])

(1, 'X', '-', 'NA', 360.0, 136.6, 0, 0.0, 49, -2.3, 0, 0.0, 2, -0.3)
(2, 'N', 'B', 0.20382165605095542, -134.2, 150.2, 47, -0.2, 4, -2.0, 1, -0.1, 49, -0.2)
(3, 'L', 'H', 0.036585365853658534, -56.5, -47.7, 47, -2.6, 4, -2.7, -2, -0.3, 5, -0.2)
(4, 'T', 'H', 0.30985915492957744, -65.1, -35.1, 1, -0.2, 4, -1.4, 2, -0.2, -1, -0.2)
(5, 'E', 'H', 0.6185567010309279, -62.3, -47.2, 1, -0.2, 3, -0.4, 2, -0.2, -1, -0.2)
(6, 'L', 'H', 0.042682926829268296, -62.3, -41.0, -4, -2.0, 3, -2.1, 1, -0.2, -2, -0.2)
(7, 'K', 'H', 0.2731707317073171, -60.4, -29.0, -4, -2.7, -1, -0.2, 1, -0.3, -2, -0.2)
(8, 'N', 'T', 0.8789808917197452, -93.9, 9.5, -4, -1.4, -1, -0.3, -3, -0.4, -2, -0.2)
(9, 'T', 'S', 0.19718309859154928, -99.6, 135.1, -3, -2.1, 5, -0.1, -5, -0.1, -3, -0.0)
(10, 'P', '-', 0.5, -57.6, 145.2, 0, 0.0, 4, -1.8, 0, 0.0, 3, -0.6)
(11, 'V', 'H', 0.4084507042253521, -57.1, -41.6, 1, -0.2, 4, -2.6, 2, -0.2, 5, -0.2)
(12, 'S', 'H', 0.6307692307692307, -61.6, -33.2, 1, -0.2, 4, -1.8, 2, -0.2, -1, -0

In [17]:
p = PDBParser()
structure = p.get_structure("1GJ7","../data/1GJ7.pdb")
model = structure[0]
DSSP = DSSP(model, "../data/1GJ7.pdb")

/home/epsilon/miniconda3/envs/projet-tut/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 4753.
  warnings.warn(
/home/epsilon/miniconda3/envs/projet-tut/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 4780.
  warnings.warn(
